In [1]:
import os,sys
from pathlib import Path
currentdir = Path(os.getcwd())
sys.path.insert(0, os.path.dirname(currentdir.parent))
sys.path.insert(0, os.path.dirname(currentdir))
print(str(sys.path[0:3]))

['c:\\Users\\anaxa\\Documents\\Projects\\BAMT', 'c:\\Users\\anaxa\\Documents\\Projects', 'c:\\Users\\anaxa\\Documents\\Projects\\BAMT\\experiments']


In [2]:
import pandas as pd
from sklearn import preprocessing
import seaborn as sns
import matplotlib.pyplot as plt

from bamt.Preprocessors import Preprocessor
from bamt.ScoringFunctions import BICGMM, BICGauss, BIC_local, BIC_local_gauss
from bamt.mi_entropy_gauss import mi_gauss
import bamt.Networks as Nets
from pgmpy.estimators import K2Score, BicScore
from pgmpy.estimators import HillClimbSearch
import networkx as nx
from bamt.utils.MathUtils import *
from gmr import GMM
from sklearn.metrics import mean_squared_error

In [3]:
data = pd.read_csv('../Data/daks_processed_with_rf_ntg.csv')
data


,Field name,Reservoir unit,Country,Region,Tectonic regime,Onshore/offshore,Hydrocarbon type,Productive area,Period,Depositional system,...,Permeability,Structural setting,Trapping mechanism,Depth,Oil density,Oil recovery factor,Condensate recovery factor,Gas recovery factor,NTG,NTG_calc
0,A.J.BERMUDEZ,UNNAMED (CUNDUACAN FIELD),MEXICO,LATIN AMERICA,COMPRESSION,ONSHORE,OIL,NaN,CRETACEOUS,FORESLOPE,...,NaN,WRENCH,FOLD,3819.8,876.160991,0.33,NaN,NaN,NaN,NaN
1,A.J.BERMUDEZ,UNNAMED (SAMARIA FIELD),MEXICO,LATIN AMERICA,COMPRESSION,ONSHORE,OIL,24700.0,CRETACEOUS,FORESLOPE,...,NaN,WRENCH,FOLD,4120.1,876.160991,0.25,NaN,NaN,NaN,NaN
2,AB TEYMUR,ILAM,IRAN,MIDDLE EAST,COMPRESSION,ONSHORE,OIL,130910.0,CRETACEOUS,LOW-ENERGY CARBONATE MUD,...,NaN,FORELAND,FOLD,3063.1,904.153355,0.13,NaN,NaN,0.650,0.668255
3,AB TEYMUR,SARVAK-1,IRAN,MIDDLE EAST,COMPRESSION,ONSHORE,OIL,133380.0,CRETACEOUS,LOW-ENERGY CARBONATE MUD,...,NaN,FORELAND,FOLD,3263.1,909.967846,0.13,NaN,NaN,0.750,0.730104
4,ABKATUN,TAMABRA,MEXICO,LATIN AMERICA,COMPRESSION,OFFSHORE,OIL,58045.0,CRETACEOUS-PALEOGENE,FORESLOPE,...,3.1,SALT,FOLD,2920.7,887.147335,0.40,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1107,ПРИОБСКОЕ,Пласт: АС 10.0-1,RUSSIA,SIBERIA,NaN,ONSHORE,OIL,NaN,CRETACEOUS,DEEP MARINE,...,23.6,NaN,NaN,2487.0,880.800000,NaN,NaN,NaN,0.765,0.159504
1108,ПРИОБСКОЕ,Пласт: АС 10.1-3,RUSSIA,SIBERIA,NaN,ONSHORE,OIL,NaN,CRETACEOUS,DEEP MARINE,...,32.7,NaN,NaN,2624.0,NaN,NaN,NaN,NaN,0.700,0.399123
1109,ПРИОБСКОЕ,Пласт: АС 10.4,RUSSIA,SIBERIA,NaN,ONSHORE,OIL,NaN,CRETACEOUS,DEEP MARINE,...,33.4,NaN,NaN,2604.0,NaN,NaN,NaN,NaN,0.685,0.209756
1110,ПРИОБСКОЕ,Пласт: АС 12.1,RUSSIA,SIBERIA,NaN,ONSHORE,OIL,NaN,CRETACEOUS,DEEP MARINE,...,28.7,NaN,NaN,2628.0,875.450000,NaN,NaN,NaN,0.755,0.958904
